In [30]:
PROJECT = "emart-datafabric"
DATASET = "common_dev"
# TABLE = "dfm_sample_eapp_data"
TABLE = "test_spark_to_redis"
DT = "2022-08-30"
LIMIT = 30

REDIS_INFO = {
    "host": "redis-master.redis-farm.svc.cluster.local",
    "port": "6379",
}

In [19]:
from pydatafabric.gcp import get_spark, bq_to_df

spark = get_spark(extra_jars="gs://emart-datafabric-resources/jars/spark-redis_2.12-3.1.0-SNAPSHOT-jar-with-dependencies.jar")
spark.conf.set("spark.redis.host", REDIS_INFO["host"])
spark.conf.set("spark.redis.port", REDIS_INFO["port"])
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

df = bq_to_df(f"""
    select  *
    from  `{PROJECT}.{DATASET}.{TABLE}`
    WHERE dt = '{DT}'
    limit {LIMIT}
""", spark_session=spark)

In [20]:
from pyspark.sql.types import StructType,StructField, StringType

columns = ['id', 'dogs', 'cats']
vals = [(1, 2, 0), (2, 0, 1)]

df = spark.createDataFrame(vals, columns)

newRow = spark.createDataFrame([(4,5,7)], columns)
df = df.union(newRow)

In [21]:
spark.conf.get("spark.jars")

'hdfs:///jars/spark-bigquery-with-dependencies_2.12-latest.jar,gs://emart-datafabric-resources/jars/spark-redis_2.12-3.1.0-SNAPSHOT-jar-with-dependencies.jar'

In [22]:
df.show(10)

+---+----+----+
| id|dogs|cats|
+---+----+----+
|  1|   2|   0|
|  2|   0|   1|
|  4|   5|   7|
+---+----+----+



In [23]:
df.createOrReplaceTempView(f"temp_{TABLE}")

In [24]:
result_df = spark.sql(
    f'select * from temp_{TABLE} limit 1')

result_df.show(10)

+---+----+----+
| id|dogs|cats|
+---+----+----+
|  1|   2|   0|
+---+----+----+



In [25]:
result_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- dogs: long (nullable = true)
 |-- cats: long (nullable = true)



In [31]:
import redis
import json

# 레디스 연결
# rd = redis.StrictRedis(host=REDIS_INFO["host"], port=REDIS_INFO["port"], db=0)

if result_df.count() > 0:
    # json_df = result_df.toJSON().collect()[0]
    # rd.set("test_dfm_sample_eapp_data", json_df)
    
    result_df.write.format("org.apache.spark.sql.redis").option("table", "test_dfm_sample_eapp_data").save()

Py4JJavaError: An error occurred while calling o319.save.
: redis.clients.jedis.exceptions.JedisConnectionException: Could not get a resource from the pool
	at redis.clients.jedis.util.Pool.getResource(Pool.java:84)
	at redis.clients.jedis.JedisPool.getResource(JedisPool.java:377)
	at com.redislabs.provider.redis.ConnectionPool$.connect(ConnectionPool.scala:35)
	at com.redislabs.provider.redis.RedisEndpoint.connect(RedisConfig.scala:94)
	at com.redislabs.provider.redis.RedisConfig.clusterEnabled(RedisConfig.scala:227)
	at com.redislabs.provider.redis.RedisConfig.getNodes(RedisConfig.scala:367)
	at com.redislabs.provider.redis.RedisConfig.getHosts(RedisConfig.scala:267)
	at com.redislabs.provider.redis.RedisConfig.<init>(RedisConfig.scala:166)
	at com.redislabs.provider.redis.RedisConfig$.fromSparkConfAndParameters(RedisConfig.scala:154)
	at org.apache.spark.sql.redis.RedisSourceRelation.<init>(RedisSourceRelation.scala:34)
	at org.apache.spark.sql.redis.DefaultSource.createRelation(Def

In [136]:
result = rd.get("test_dfm_sample_eapp_data")
result = result.decode('utf-8')
result = dict(json.loads(result))

print(result)

{'review_id': '2208300923271031', 'cust_id': '03c4f22b4d5b998916c8ae5358c68f9d1b89ff96a93bc4d723518ba7bcf3f0dd', 'gender_cd': '1', 'gender_nm': '남', 'age': '36', 'agrde_cd_10_unit': '30', 'store_cd': '1000', 'store_nm': '이마트 창동점', 'score': 10, 'comments': '저렴하고 맛있어요!!!!!!!!!', 'comments_length': 18, 'sku_cd': '8809676682501', 'sku_nm': '(SF)피코크 된장찌개 요리재료', 'order_dt': '2022-08-29', 'dt': '2022-08-30', 'prdt_cat_cd': '13', 'prdt_cat_nm': '채소', 'prdt_di_cd': '10', 'prdt_di_nm': '신선1담당', 'prdt_gcode_cd': '130', 'prdt_gcode_nm': '간편채소', 'prdt_mcode_cd': '0438', 'prdt_mcode_nm': 'PEACOCK밀키트', 'prdt_dcode_cd': '0263', 'prdt_dcode_nm': 'P)한식(국탕)밀키트', 'area_cd': '00', 'area_nm': '서울', 'longitude': 127.046690683, 'latitude': 37.651608368, 'comments_point': 10, 'image_point': 10, 'thumb_point': 0, 'action_cd': '003', 'blind_flag': 'N', 'active_flag': 'Y', 'tag_list': '130::1::10,130::2::10,130::3::10,130::4::10,130::5::10', 'avg_tag_score': 10.0}


In [13]:
spark.stop()